The ideia is compute the mean error in the density profile as defined in Li+16, Eq. (20)

In [3]:
import os
import sys
from optparse import OptionParser

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy import interpolate
import pickle
from scipy.stats.mstats import mquantiles
import json

from scipy import integrate


In [70]:
def restrict_density(density, r, r1, r2):
    """
        Return the density and the radius between r1 and r2.
        r1 < r2
    """
    assert r1 < r2, "r2 should be greater than r1."
    
    i = (r1 < r) &  (r < r2)
    
    return density[i], r[i]


def mean_density_error(density_true, density_model, r_true, r_model, r1, r2):
    
    true_  = interpolate.interp1d(r_true, density_true, kind="cubic")
    model_ = interpolate.interp1d(r_model, density_model, kind="cubic")
    
    r_ = np.linspace(r1, r2, num=100)
    
    true_interp  = np.asarray(true_(r_))
    model_interp = np.asarray(model_(r_))
    
    print(true_interp)
    
    integral = np.absolute( model_interp - true_interp ) / true_interp
    
   
    return integrate.trapezoid(y=integral, x=r_) / r2
    


In [71]:
data_path = "/home/carlosmelo/Documents/GitHub/Illustris/my_illustris/TNG50-1-snap84-subhalo9/"
log_img = fits.open(data_path+"/imgs/log_img.fits")[1].data
distance = log_img["distance"]
pc       = distance*np.pi/0.648  # Constant factor to convert arcsec --> pc

    # Load DM density profile
dm_hdu = fits.open(data_path+"/dm/density_fit.fits")
rho_dm = dm_hdu[1].data["density"]
r_dm   = dm_hdu[1].data["radius"]

    # Load star density profile
star_hdu = fits.open(data_path+"/imgs/stellar_density.fits")
rho_star = star_hdu[1].data["density"]
r_star   = star_hdu[1].data["radius"]


true_total = rho_dm + rho_star
r_true = r_dm*1e3

In [72]:
result_path = "./9/model1/normal/JAM/Fiducial/Analysis_2.5Reff/"

with open(result_path+f'/densities/Fiducial_stellar_density3D.pickle','rb') as f:
    stellar_density = pickle.load(f)
    f.close()
            
with open(result_path+f'/densities/Fiducial_DM_density3D.pickle','rb') as f:
    dm_density = pickle.load(f)
    f.close()

with open(result_path+"/quantities.json") as f:
        quantities = json.load(f)
        f.close()
        
with open(result_path+"/description.json") as f:
        description = json.load(f)
        f.close()

reff = quantities["Reff"]
# Load DM density profile
model_dm = dm_density["distribution"]

# Load star density profile
model_star =stellar_density["distribution"]
r   = stellar_density["radii"]

# Total mass density profile
model_total = model_dm + model_star
r_model   = r * pc


"""rho_dm = mquantiles(dm_density_dist["distribution"], 0.5, axis=0)[0]
r_dm   = dm_density_dist["radii"]

# Load star density profile
rho_star = mquantiles(stellar_density_dist["distribution"], 0.5, axis=0)[0]
r_star   = stellar_density_dist["radii"]

# Total mass density profile
rho_total = rho_dm + rho_star
r_total   = r_dm"""

'rho_dm = mquantiles(dm_density_dist["distribution"], 0.5, axis=0)[0]\nr_dm   = dm_density_dist["radii"]\n\n# Load star density profile\nrho_star = mquantiles(stellar_density_dist["distribution"], 0.5, axis=0)[0]\nr_star   = stellar_density_dist["radii"]\n\n# Total mass density profile\nrho_total = rho_dm + rho_star\nr_total   = r_dm'

In [73]:
description["Ddensity"] = "Mean error of the density profile"
out_descripition = open("{}/description.json".format(result_path), "w")
json.dump(description, out_descripition, indent = 8)
out_descripition.close()


In [74]:
r1 = r_model.min()
r2 = 2.5*reff*pc



In [113]:
true_  = interpolate.interp1d(r_true, true_total, kind="cubic")
model_ = interpolate.interp1d(r_model, model_total, kind="cubic")

r_ = np.linspace(r1, r2, 25)


In [114]:
r_.shape

(25, 1)

In [98]:



true_interp  = np.asarray(true_(r_))
model_interp = np.asarray(model_(r_))

integral = np.absolute( model_interp - true_interp ) / true_interp

integrate.trapezoid(y=integral, x=r_)/r2

array([0.02678543])

In [75]:
mean_error = mean_density_error(density_true=true_total, density_model=model_total,
                                   r_true=r_true, r_model=r_model, r1=r1, r2=r2)

[[2.48799968]
 [1.96335125]
 [1.59131373]
 [1.25440115]
 [1.03965562]
 [0.8532542 ]
 [0.71788839]
 [0.62061647]
 [0.52386211]
 [0.4570183 ]
 [0.38464344]
 [0.34353033]
 [0.31275396]
 [0.28218629]
 [0.2469365 ]
 [0.21866094]
 [0.20280123]
 [0.191104  ]
 [0.16996961]
 [0.15591225]
 [0.14665639]
 [0.13360141]
 [0.12556752]
 [0.11513818]
 [0.10796157]
 [0.10329013]
 [0.0965419 ]
 [0.08912565]
 [0.08236227]
 [0.07797587]
 [0.07731247]
 [0.075802  ]
 [0.07121761]
 [0.06758578]
 [0.06557403]
 [0.06368716]
 [0.06108509]
 [0.05776269]
 [0.05505549]
 [0.05351213]
 [0.0524234 ]
 [0.05113524]
 [0.04917595]
 [0.04687008]
 [0.04528435]
 [0.04443044]
 [0.04308592]
 [0.0408907 ]
 [0.03901877]
 [0.03830912]
 [0.03813849]
 [0.03768131]
 [0.03670393]
 [0.03525416]
 [0.03355154]
 [0.0320433 ]
 [0.03108563]
 [0.03045652]
 [0.02974341]
 [0.02879335]
 [0.02787561]
 [0.02727191]
 [0.02691456]
 [0.02648781]
 [0.02573283]
 [0.02479316]
 [0.02397263]
 [0.02349938]
 [0.02324228]
 [0.02296425]
 [0.02247591]
 [0.02

In [76]:
mean_error

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])